In [1]:
# Just for the notebooks, the cwd needs to be set to the root of the project
import os
from pathlib import Path
home = Path.home()

cwd = os.getcwd()
if not 'initial_cwd' in locals():
	initial_cwd = cwd

# check if any of the parent directories is 'notebooks'
relative_path = Path(cwd).relative_to(home)
if 'notebooks' in relative_path.parts:
	# if so, change the current working directory to the root of the project
	while relative_path.parts and not 'notebooks' in os.listdir(cwd):
		cwd = os.path.dirname(cwd)
		relative_path = Path(cwd).relative_to(home)
	if 'notebooks' in os.listdir(cwd):
		os.chdir(cwd)

print(f"Current working directory: {os.getcwd()}")
print(f"Initial working directory: {initial_cwd}")

Current working directory: /home/cyphix/repos/pixel-ota
Initial working directory: /home/cyphix/repos/pixel-ota/notebooks


In [2]:
%pip install -r requirements.txt --quiet
from dotenv import load_dotenv
load_dotenv()


Note: you may need to restart the kernel to use updated packages.


False

# Setup logging

In [3]:

import logging
from LoggingColor import ColorHandler
import sys
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(name)-16s [%(levelname)-1s]: %(message)s',
    handlers=[
        logging.FileHandler(f'log.log',mode='w'),
        #logging.StreamHandler(sys.stdout),
        ColorHandler(sys.stdout)
    ],
    datefmt='%H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.debug("Debug log test")
logger.info("Info log test")
logger.warning("Warning log test")
logger.error("Error log test")
logger.critical("Critical log test")

18:57:18.230 __main__         [D]: Debug log test
18:57:18.230 __main__         [I]: Info log test
18:57:18.230 __main__         [W]: Warning log test
18:57:18.230 __main__         [E]: Error log test
18:57:18.230 __main__         [C]: Critical log test


# Fetch ota links

In [20]:
from dataclasses import dataclass, field

@dataclass
class OTAInfo:
    android_version: str
    buildID: str
    build_branch: str
    build_date: str
    build_number: str
    build_variant: str | None
    carrier: str | None
    device: str
    url: str
    checksum: str

    def download(self, download_dir: str, filename: str = None, overwrite: bool = False) -> str:
        import requests
        import os
        from tqdm.auto import tqdm

        if filename is None:
            filename = f"{self.device}-ota-{self.buildID}.zip"

        os.makedirs(download_dir, exist_ok=True)
        out_path = os.path.join(download_dir, filename)

        if not overwrite and os.path.exists(out_path):
            logger.warning(f"File {out_path} already exists, skipping download.")
        else:
            logger.info(f"Downloading OTA {self.android_version}, {self.buildID} for {self.device} to {out_path}")
            temp_path = out_path + ".part"
            with requests.get(self.url, stream=True) as r:
                total_size = int(r.headers.get('content-length', 0))
                with tqdm(total=total_size, unit='B', unit_scale=True, desc=filename) as pbar:
                    with open(temp_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                            pbar.update(len(chunk))
                assert total_size == os.path.getsize(temp_path), f"Downloaded file size {os.path.getsize(temp_path)} does not match expected size {total_size}"

            os.rename(temp_path, out_path)
        
        # verify checksum
        import hashlib
        sha256 = hashlib.sha256()
        with open(out_path, 'rb') as f:
            with tqdm(total=os.path.getsize(out_path), unit='B', unit_scale=True, desc="Verifying checksum") as pbar:
                for chunk in iter(lambda: f.read(8192), b''):
                    sha256.update(chunk)
                    pbar.update(len(chunk))
        calculated_checksum = sha256.hexdigest()
        if calculated_checksum != self.checksum:
            logger.error(f"Checksum mismatch for {out_path}: expected {self.checksum}, got {calculated_checksum}")
            os.remove(out_path)
            raise ValueError(f"Checksum mismatch for {out_path}: expected {self.checksum}, got {calculated_checksum}")
        else:
            logger.info(f"Checksum verified for {out_path}")
        return out_path

def fetchAllOTA() -> list[OTAInfo]:
    import requests
    import re

    with requests.Session() as s:
        cookies = {
            "devsite_wall_acks": "nexus-ota-tos",
        }

        s.cookies.update(cookies)
        res = s.get("https://developers.google.com/android/ota")

    assert res.status_code == 200, f"Failed to fetch OTA page: {res.status_code}"

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(res.text, 'html.parser')

    # find all table rows
    rows = soup.find_all('tr')
    logger.info(f"Found {len(rows)} rows in the OTA table")

    available_otas = []
    for row in rows:
        # There should be 3 columns: Version, Download, and checksum
        cols = row.find_all('td')

        if len(cols) != 3:
            logger.warning(f"Skipping row with {len(cols)} columns")
            continue
        version_col, download_col, checksum_col = cols

        # extract the version data
        version_text = version_col.get_text(strip=True)
        match = re.match(r'^(\d+\.\d+\.\d+)\s+\(([^,]+),\s+([^,]+)(?:,\s+([^)]+))?\)$', version_text)
        if not match:
            logger.warning(f"Skipping row with unrecognized version format: {version_text}")
            continue
        android_ver, build_id, security_level, carrier  = match.groups()

        split_build_id = build_id.split('.')
        try:
            build_branch_code, build_date, build_number = split_build_id[:3]
        except Exception as e:
            logger.error(f"Error parsing build ID '{build_id}' (most likely due to old version format): {e}")
            continue
        build_variant = split_build_id[3] if len(split_build_id) > 3 else None

        # get the device name from the row using the build_id
        row_id = row.get('id')
        # the format is <device>-<build_id>
        device = row_id.lower().replace(f"{build_id.lower()}", "")

        logger.info(f"Parsed version: Android {android_ver}, Build {build_id}, Security {security_level}, Carrier {carrier}")
        logger.debug(f"Device: {device}")
        logger.debug(f"Build details: Branch {build_branch_code}, Date {build_date}, Number {build_number}, Variant {build_variant}")


        # extract the download link
        a_tag = download_col.find('a', href=True)
        if not a_tag:
            logger.warning("Skipping row with no download link")
            continue
        dl_link = a_tag['href']
        logger.info(f"Found download link: {dl_link}")

        checksum_text = checksum_col.get_text(strip=True)
        logger.info(f"Checksum: {checksum_text}")
        ota_info = OTAInfo(
            android_version=android_ver,
            buildID=build_id,
            build_branch=build_branch_code,
            build_date=build_date,
            build_number=build_number,
            build_variant=build_variant,
            carrier=carrier,
            device=device,
            url=dl_link,
            checksum=checksum_text
        )
        available_otas.append(ota_info)

    return available_otas

otas = fetchAllOTA()
logger.info(f"Fetched {len(otas)} OTA entries")

19:13:17.664 urllib3.connectionpool [D]: Starting new HTTPS connection (1): developers.google.com:443
19:13:18.612 urllib3.connectionpool [D]: https://developers.google.com:443 "GET /android/ota HTTP/1.1" 200 None
19:13:19.151 __main__         [I]: Found 1696 rows in the OTA table
19:13:19.151 __main__         [I]: Parsed version: Android 16.0.0, Build BD1A.250702.001, Security Aug 2025, Carrier None
19:13:19.151 __main__         [D]: Device: blazer
19:13:19.151 __main__         [D]: Build details: Branch BD1A, Date 250702, Number 001, Variant None
19:13:19.151 __main__         [I]: Found download link: https://dl.google.com/dl/android/aosp/blazer-ota-bd1a.250702.001-23331fee.zip
19:13:19.151 __main__         [I]: Checksum: 23331feef312884bac74cc5d058f4c30d017443d2e984168250c550ac3883151
19:13:19.151 __main__         [I]: Parsed version: Android 16.0.0, Build BD1A.250702.001.A3, Security Aug 2025, Carrier Verizon
19:13:19.151 __main__         [D]: Device: blazer
19:13:19.151 __main__  

In [21]:
import pandas as pd
ota_df = pd.DataFrame(otas)
ota_df['obj'] = otas
filtered_releases = ota_df[ota_df['device'] == 'lynx'] # I only want releases for the pixel 7a
filtered_releases = filtered_releases[filtered_releases['carrier'].isnull()] # I only want releases for the unlocked version
filtered_releases.sort_values(by=['build_date', 'build_number'], ascending=[False, False], inplace=True)
filtered_releases

selected_ota = filtered_releases.iloc[0].obj
download_dir = os.path.join(os.getcwd(), 'downloads')
selected_ota.download(download_dir=download_dir, overwrite=False)


19:13:20.830 __main__         [W]: File /home/cyphix/repos/pixel-ota/downloads/lynx-ota-BP3A.250905.014.zip already exists, skipping download.


Verifying checksum:   0%|          | 0.00/2.89G [00:00<?, ?B/s]

19:13:22.794 __main__         [I]: Checksum verified for /home/cyphix/repos/pixel-ota/downloads/lynx-ota-BP3A.250905.014.zip


'/home/cyphix/repos/pixel-ota/downloads/lynx-ota-BP3A.250905.014.zip'